In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Navigate to your project directory in Google Drive
%cd /content/drive/MyDrive/CAF-GAN/

# Unzip the image files (this might take a few minutes)
# The -q makes the output quiet, -n prevents unzipping if already done
!unzip -q -n mimic-cxr-jpg-2.0.0.zip

print("✅ Workspace ready and images unzipped.")

/content/drive/MyDrive/CAF-GAN
✅ Workspace ready and images unzipped.


In [2]:
!pip install pyyaml pandas scikit-learn albumentations segmentation-models-pytorch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.3 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    """
    CAF-GAN Generator Network (DCGAN architecture).
    Takes a latent vector z and outputs a 256x256 grayscale image.
    Output is normalized between -1 and 1 using Tanh.
    """
    def __init__(self, latent_dim, channels=1):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.main = nn.Sequential(
            # Input: latent_dim x 1 x 1
            nn.ConvTranspose2d(latent_dim, 1024, 4, 1, 0, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
            # State: 1024 x 4 x 4
            nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            # State: 512 x 8 x 8
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # State: 256 x 16 x 16
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # State: 128 x 32 x 32
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # State: 64 x 64 x 64
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            # State: 32 x 128 x 128
            nn.ConvTranspose2d(32, channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: channels x 256 x 256
        )

    def forward(self, input):
        return self.main(input)

In [4]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    """
    CAF-GAN Discriminator Network (Critic for WGAN-GP).
    Takes a 256x256 grayscale image and outputs a single scalar score.
    Uses InstanceNorm instead of BatchNorm for stability with WGAN-GP.
    No final activation function.
    """
    def __init__(self, channels=1):
        super(Discriminator, self).__init__()

        self.main = nn.Sequential(
            # Input: channels x 256 x 256
            nn.Conv2d(channels, 32, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State: 32 x 128 x 128
            nn.Conv2d(32, 64, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(64, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # State: 64 x 64 x 64
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(128, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # State: 128 x 32 x 32
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(256, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # State: 256 x 16 x 16
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(512, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # State: 512 x 8 x 8
            nn.Conv2d(512, 1024, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(1024, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # State: 1024 x 4 x 4
            nn.Conv2d(1024, 1, 4, 1, 0, bias=False)
            # Output: 1 x 1 x 1
        )

    def forward(self, input):
        return self.main(input)

In [5]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

# This single dataset file will serve both critic training scripts.

class MIMICCXRClassifierDataset(Dataset):
    """
    Dataset for the Cdiag (classification) task.
    - Loads a JPG image.
    - Converts it to RGB (as required by ResNet).
    - Returns the image and its corresponding Pneumonia label.
    """
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        subject_id = str(row['subject_id'])
        study_id = str(row['study_id'])
        dicom_id = row['dicom_id']

        # Construct the path to the JPG image
        image_path = os.path.join(
            self.image_dir, f'p{subject_id[:2]}', f'p{subject_id}', f's{study_id}', f'{dicom_id}.jpg'
        )

        # Load image and convert to a numpy array in RGB format
        image = Image.open(image_path).convert("RGB")
        image = np.array(image)

        # Apply augmentations
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        # Get the label
        label = torch.tensor(row['Pneumonia'], dtype=torch.float32)

        return image, label.unsqueeze(0)


class MIMICXRSegmentationDataset(Dataset):
    """
    Dataset for the Cseg (segmentation) task.
    - Loads a JPG image (as 3-channel RGB). <--- UPDATED
    - Loads its corresponding pre-generated PNG mask.
    - Returns both the image and the mask.
    """
    def __init__(self, df, image_dir, mask_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        subject_id = str(row['subject_id'])
        study_id = str(row['study_id'])
        dicom_id = row['dicom_id']

        image_path = os.path.join(
            self.image_dir, f'p{subject_id[:2]}', f'p{subject_id}', f's{study_id}', f'{dicom_id}.jpg'
        )
        mask_path = os.path.join(self.mask_dir, f"{dicom_id}.png")

        # --- KEY CHANGE ---
        # Load image and convert to RGB to match the model's expected input channels.
        image = np.array(Image.open(image_path).convert("RGB"), dtype=np.float32)

        # Load mask as grayscale
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)

        # RESIZE THE IMAGE to match the mask size (256×256) BEFORE augmentation
        image = np.array(Image.fromarray(image.astype(np.uint8)).resize((256, 256), Image.BILINEAR))

        # Normalize mask values from [0, 255] to [0.0, 1.0]
        mask[mask == 255.0] = 1.0

        # Apply augmentations (Albumentations will now see matching input sizes from the Resize transform)
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # Add a channel dimension for the mask for consistency
        return image, mask.unsqueeze(0)

class MIMICCXR_GANDataset(Dataset):
    """
    Dataset for the main GAN training.
    - Loads a JPG image (as grayscale).
    - Returns the image, its Pneumonia label, and the one-hot encoded race group.
    """
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

        # Pre-process sensitive attributes
        self.df['race_group'] = self.df['race_group'].astype('category')
        self.race_categories = self.df['race_group'].cat.categories
        self.one_hot_races = pd.get_dummies(self.df['race_group'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        subject_id = str(row['subject_id'])
        study_id = str(row['study_id'])
        dicom_id = row['dicom_id']

        image_path = os.path.join(
            self.image_dir, f'p{subject_id[:2]}', f'p{subject_id}', f's{study_id}', f'{dicom_id}.jpg'
        )

        # Load the image and convert to "RGB"
        image = Image.open(image_path).convert("RGB") # <-- CHANGE "L" to "RGB"
        image = np.array(image, dtype=np.float32)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        label = torch.tensor(row['Pneumonia'], dtype=torch.float32)
        race = torch.tensor(self.one_hot_races.iloc[idx].values, dtype=torch.float32)

        return image, label, race

In [6]:
CONFIG = {
    # --- Data Paths ---
    "IMAGE_DIR": "/content/drive/MyDrive/CAF-GAN/mimic-cxr-jpg-2.0.0/files/",
    "TRAIN_CSV_PATH": "/content/drive/MyDrive/CAF-GAN/data/splits/train.csv",
    "CDIAG_PATH": "/content/drive/MyDrive/CAF-GAN/outputs/cdiag/best_cdiag_colab.pth",
    "CSEG_PATH": "/content/drive/MyDrive/CAF-GAN/outputs/cseg/best_cseg_colab.pth",

    # --- Output Paths ---
    "OUTPUT_DIR": "/content/drive/MyDrive/CAF-GAN/outputs/gan/",
    "IMAGE_OUTPUT_DIR": "/content/drive/MyDrive/CAF-GAN/outputs/gan/generated_images/",

    # --- Model & Training Hyperparameters ---
    "IMG_SIZE": 256,
    "CHANNELS": 3,
    "LATENT_DIM": 128,
    "BATCH_SIZE": 16,  # GANs are memory intensive
    "EPOCHS": 200,     # GANs need many epochs to converge
   "G_LR": 0.0001,       # ⬅️ Optimized for your critic strengths
    "D_LR": 0.0001,       # ⬅️ Balanced with generator
    "B1": 0.5,
    "B2": 0.999,
    "N_CRITIC": 5,        # ⬅️ Standard WGAN-GP value

    # --- Loss Weights (CALIBRATED for your critic performance) ---
    "LAMBDA_GP": 10,
    "LAMBDA_FAIR": 0.15,   # ⬅️ Increased slightly (Cdiag: 66% accuracy)
    "LAMBDA_CLINIC": 0.08, # ⬅️ Increased slightly (Cseg: 57.6% Dice)

    # --- Adaptive Training Parameters ---
    "USE_GRADIENT_CLIPPING": True,
    "MAX_GRAD_NORM": 1.0,
    "LR_DECAY_FACTOR": 0.8,
    "LR_DECAY_PATIENCE": 10,

    # --- System & Logging ---
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "NUM_WORKERS": 2,
    "LOG_FREQ": 50,
    "SAVE_IMG_FREQ": 1,
    "SAVE_MODEL_FREQ": 10
}


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torchvision.utils as vutils
import pandas as pd
from tqdm import tqdm
import os
import yaml
import numpy as np
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import hashlib
import shutil

# --- Utility Functions ---
def weights_init(m):
    """Custom weights initialization called on netG and netD."""
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def gradient_penalty(critic, real, fake, device):
    """Calculates the gradient penalty for WGAN-GP."""
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)

    # Create interpolated images WITH gradient tracking
    interpolated_images = real * alpha + fake * (1 - alpha)
    interpolated_images.requires_grad_(True)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images)

    # Compute gradients
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]

    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gp = torch.mean((gradient_norm - 1) ** 2)
    return gp

def is_valid_checkpoint(file_path):
    """Check if a checkpoint file is valid."""
    if not os.path.exists(file_path):
        return False

    try:
        checkpoint = torch.load(file_path, map_location='cpu', weights_only=False)
        required_keys = ['epoch', 'netG_state_dict', 'netD_state_dict',
                        'optimizerG_state_dict', 'optimizerD_state_dict',
                        'G_losses', 'D_losses']

        if not all(key in checkpoint for key in required_keys):
            return False

        return True
    except Exception as e:
        return False

def save_checkpoint(epoch, netG, netD, optimizerG, optimizerD, G_losses, D_losses, path):
    """Save training checkpoint with integrity verification."""
    try:
        temp_path = path + ".temp"

        torch.save({
            'epoch': epoch,
            'netG_state_dict': netG.state_dict(),
            'netD_state_dict': netD.state_dict(),
            'optimizerG_state_dict': optimizerG.state_dict(),
            'optimizerD_state_dict': optimizerD.state_dict(),
            'G_losses': G_losses,
            'D_losses': D_losses
        }, temp_path)

        if is_valid_checkpoint(temp_path):
            if os.path.exists(path):
                os.remove(path)
            shutil.move(temp_path, path)
            print(f"✅ Checkpoint saved successfully for epoch {epoch}")
            return True
        else:
            if os.path.exists(temp_path):
                os.remove(temp_path)
            return False
    except Exception as e:
        if os.path.exists(temp_path):
            os.remove(temp_path)
        return False

def load_checkpoint(path, netG, netD, optimizerG, optimizerD, device):
    """Load training checkpoint with error handling."""
    if not os.path.exists(path):
        return 0, [], []

    if not is_valid_checkpoint(path):
        corrupted_path = path + ".corrupted"
        if os.path.exists(path):
            shutil.move(path, corrupted_path)
        return 0, [], []

    try:
        checkpoint = torch.load(path, map_location=device, weights_only=False)
        netG.load_state_dict(checkpoint['netG_state_dict'])
        netD.load_state_dict(checkpoint['netD_state_dict'])
        optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
        optimizerD.load_state_dict(checkpoint['optimizerD_state_dict'])
        print(f"✅ Checkpoint loaded successfully from epoch {checkpoint['epoch']}")
        return checkpoint['epoch'], checkpoint['G_losses'], checkpoint['D_losses']

    except Exception as e:
        corrupted_path = path + ".corrupted"
        if os.path.exists(path):
            shutil.move(path, corrupted_path)
        return 0, [], []

# --- 🚀 Main Training Execution ---
def run_gan_training(CONFIG):
    DEVICE = CONFIG['DEVICE']
    os.makedirs(CONFIG['IMAGE_OUTPUT_DIR'], exist_ok=True)
    os.makedirs(CONFIG['OUTPUT_DIR'], exist_ok=True)
    print(f"Using device: {DEVICE}")

    # --- Data Loading ---
    transform = A.Compose([
        A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2(),
    ])
    train_df = pd.read_csv(CONFIG['TRAIN_CSV_PATH'])
    train_dataset = MIMICCXR_GANDataset(train_df, CONFIG['IMAGE_DIR'], transform=transform)
    dataloader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True,
                           num_workers=CONFIG['NUM_WORKERS'])

    # --- Initialize Models ---
    netG = Generator(CONFIG['LATENT_DIM'], CONFIG['CHANNELS']).to(DEVICE)
    netD = Discriminator(CONFIG['CHANNELS']).to(DEVICE)

    # --- Load Frozen Critics ---
    Cdiag = models.resnet50()
    Cdiag.fc = nn.Linear(Cdiag.fc.in_features, 1)
    Cdiag.load_state_dict(torch.load(CONFIG['CDIAG_PATH'], map_location=DEVICE))
    Cdiag.eval()
    for param in Cdiag.parameters():
        param.requires_grad = False
    Cdiag.to(DEVICE)

    Cseg = smp.Unet(encoder_name="resnet34", in_channels=3, classes=1)
    Cseg.load_state_dict(torch.load(CONFIG['CSEG_PATH'], map_location=DEVICE))
    Cseg.eval()
    for param in Cseg.parameters():
        param.requires_grad = False
    Cseg.to(DEVICE)

    # --- Optimizers ---
    optimizerD = optim.Adam(netD.parameters(), lr=CONFIG['D_LR'], betas=(CONFIG['B1'], CONFIG['B2']))
    optimizerG = optim.Adam(netG.parameters(), lr=CONFIG['G_LR'], betas=(CONFIG['B1'], CONFIG['B2']))

    # --- Load from Epoch 60 ---
    checkpoint_path = f"{CONFIG['OUTPUT_DIR']}/training_checkpoint.pth"
    netG_weights_path = f"{CONFIG['OUTPUT_DIR']}/netG_epoch_60.pth"
    netD_weights_path = f"{CONFIG['OUTPUT_DIR']}/netD_epoch_60.pth"

    if os.path.exists(checkpoint_path) and is_valid_checkpoint(checkpoint_path):
        start_epoch, G_losses, D_losses = load_checkpoint(checkpoint_path, netG, netD, optimizerG, optimizerD, DEVICE)
        print(f"Resuming from epoch {start_epoch}")
    elif os.path.exists(netG_weights_path) and os.path.exists(netD_weights_path):
        print("📦 Loading weights from epoch 60")
        netG.load_state_dict(torch.load(netG_weights_path, map_location=DEVICE))
        netD.load_state_dict(torch.load(netD_weights_path, map_location=DEVICE))
        start_epoch = 60
        G_losses, D_losses = [], []
    else:
        print("🚀 Starting from scratch")
        netG.apply(weights_init)
        netD.apply(weights_init)
        start_epoch = 0
        G_losses, D_losses = [], []

    print(f"✅ Starting from epoch {start_epoch}")

    fixed_noise = torch.randn(64, CONFIG['LATENT_DIM'], 1, 1, device=DEVICE)

    # --- Training Variables ---
    best_wasserstein = float('inf')
    lr_decay_counter = 0

    # --- 🏟️ The Grand Training Loop ---
    print("Starting Training Loop...")

    for epoch in range(start_epoch, CONFIG['EPOCHS']):
        try:
            epoch_g_losses = []
            epoch_d_losses = []
            epoch_gp_values = []
            epoch_wasserstein_values = []

            for i, (real_imgs, labels, races) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{CONFIG['EPOCHS']}")):
                real_imgs, labels, races = real_imgs.to(DEVICE), labels.to(DEVICE), races.to(DEVICE)

                # ---------------------------
                # Train Discriminator (Critic)
                # ---------------------------
                netD.zero_grad()

                # Generate fake images
                noise = torch.randn(real_imgs.size(0), CONFIG['LATENT_DIM'], 1, 1, device=DEVICE)
                with torch.no_grad():
                    fake_imgs = netG(noise)

                # Real images forward pass
                critic_real = netD(real_imgs).reshape(-1)
                # Fake images forward pass
                critic_fake = netD(fake_imgs).reshape(-1)

                # Calculate Wasserstein distance
                wasserstein_dist = torch.mean(critic_real) - torch.mean(critic_fake)
                epoch_wasserstein_values.append(wasserstein_dist.item())

                # Gradient penalty
                gp = gradient_penalty(netD, real_imgs, fake_imgs, DEVICE)

                # Discriminator loss
                loss_critic = -wasserstein_dist + CONFIG['LAMBDA_GP'] * gp
                loss_critic.backward()

                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(netD.parameters(), max_norm=1.0)
                optimizerD.step()

                # Update generator every N_CRITIC steps
                if i % CONFIG['N_CRITIC'] == 0:
                    # ---------------------
                    # Train Generator
                    # ---------------------
                    netG.zero_grad()

                    # Generate fresh fake images
                    noise = torch.randn(real_imgs.size(0), CONFIG['LATENT_DIM'], 1, 1, device=DEVICE)
                    fake_imgs = netG(noise)

                    # 1. Adversarial Loss
                    critic_output = netD(fake_imgs)
                    adversarial_loss = -torch.mean(critic_output)

                    # 2. Fairness Loss
                    with torch.no_grad():
                        diag_output = Cdiag(fake_imgs)
                        diag_preds = torch.sigmoid(diag_output).squeeze()

                    fairness_loss = 0.0
                    tpr_per_group = []
                    for j in range(races.shape[1]):
                        group_mask = races[:, j] == 1
                        if group_mask.sum() > 0:
                            tpr_num = torch.sum(diag_preds[group_mask] * labels[group_mask])
                            tpr_den = torch.sum(labels[group_mask]) + 1e-6
                            tpr_per_group.append(tpr_num / tpr_den)

                    if len(tpr_per_group) > 1:
                        tpr_tensor = torch.stack(tpr_per_group)
                        if not torch.isnan(tpr_tensor).any():
                            fairness_loss = torch.std(tpr_tensor)  # Use standard deviation for better fairness

                    # 3. Clinical Loss
                    with torch.no_grad():
                        seg_masks = torch.sigmoid(Cseg(fake_imgs))
                        clinical_loss = torch.mean((1 - seg_masks) ** 2)

                    # Total Generator Loss
                    loss_gen = (adversarial_loss +
                               CONFIG['LAMBDA_FAIR'] * fairness_loss +
                               CONFIG['LAMBDA_CLINIC'] * clinical_loss)

                    loss_gen.backward()

                    # Gradient clipping
                    torch.nn.utils.clip_grad_norm_(netG.parameters(), max_norm=1.0)
                    optimizerG.step()

                    # Store losses for logging
                    G_losses.append(loss_gen.item())
                    D_losses.append(loss_critic.item())
                    epoch_g_losses.append(loss_gen.item())
                    epoch_d_losses.append(loss_critic.item())
                    epoch_gp_values.append(gp.item())

                # --- Logging ---
                if i % CONFIG['LOG_FREQ'] == 0:
                    avg_g_loss = np.mean(epoch_g_losses[-10:]) if epoch_g_losses else 0
                    avg_d_loss = np.mean(epoch_d_losses[-10:]) if epoch_d_losses else 0
                    avg_gp = np.mean(epoch_gp_values[-10:]) if epoch_gp_values else 0
                    avg_wasserstein = np.mean(epoch_wasserstein_values[-10:]) if epoch_wasserstein_values else 0

                    print(f"[{epoch+1}/{CONFIG['EPOCHS']}][{i}/{len(dataloader)}] "
                          f"Loss_D: {avg_d_loss:.4f} "
                          f"Loss_G: {avg_g_loss:.4f} "
                          f"GP: {avg_gp:.4f} "
                          f"W_dist: {avg_wasserstein:.4f}")

                # --- Save checkpoint every 20 batches ---
                if i % 20 == 0:
                    save_checkpoint(epoch, netG, netD, optimizerG, optimizerD, G_losses, D_losses, checkpoint_path)

            # --- Validation & Monitoring ---
            # Calculate epoch statistics
            epoch_avg_g_loss = np.mean(epoch_g_losses) if epoch_g_losses else 0
            epoch_avg_d_loss = np.mean(epoch_d_losses) if epoch_d_losses else 0
            epoch_avg_wasserstein = np.mean(epoch_wasserstein_values) if epoch_wasserstein_values else 0

            print(f"\n📊 Epoch {epoch+1} Summary:")
            print(f"   Avg G Loss: {epoch_avg_g_loss:.4f}")
            print(f"   Avg D Loss: {epoch_avg_d_loss:.4f}")
            print(f"   Avg Wasserstein: {epoch_avg_wasserstein:.4f}")

            # Learning rate scheduling
            if epoch_avg_wasserstein < best_wasserstein:
                best_wasserstein = epoch_avg_wasserstein
                lr_decay_counter = 0
            else:
                lr_decay_counter += 1
                if lr_decay_counter >= 5:  # Patience of 5 epochs
                    for param_group in optimizerG.param_groups:
                        param_group['lr'] *= 0.8
                    for param_group in optimizerD.param_groups:
                        param_group['lr'] *= 0.8
                    lr_decay_counter = 0
                    print(f"📉 Learning rate reduced to: {optimizerG.param_groups[0]['lr']}")

            # --- Save Images ---
            if (epoch + 1) % CONFIG['SAVE_IMG_FREQ'] == 0:
                with torch.no_grad():
                    fake_grid = netG(fixed_noise).detach().cpu()
                vutils.save_image(fake_grid, f"{CONFIG['IMAGE_OUTPUT_DIR']}/epoch_{epoch+1}.png",
                                 normalize=True, nrow=8)
                print(f"🖼️  Saved generated images for epoch {epoch+1}")

            # --- Save model checkpoints ---
            if (epoch + 1) % CONFIG.get('SAVE_MODEL_FREQ', 10) == 0:
                torch.save(netG.state_dict(), f"{CONFIG['OUTPUT_DIR']}/netG_epoch_{epoch+1}.pth")
                torch.save(netD.state_dict(), f"{CONFIG['OUTPUT_DIR']}/netD_epoch_{epoch+1}.pth")
                print(f"💾 Saved model weights for epoch {epoch+1}")

            # --- Save checkpoint at end of epoch ---
            save_checkpoint(epoch, netG, netD, optimizerG, optimizerD, G_losses, D_losses, checkpoint_path)
            print(f"💾 Epoch {epoch+1} completed. Checkpoint saved.")

        except Exception as e:
            print(f"❌ Error occurred during epoch {epoch+1}: {e}")
            emergency_path = f"{CONFIG['OUTPUT_DIR']}/emergency_checkpoint_epoch_{epoch}.pth"
            save_checkpoint(epoch, netG, netD, optimizerG, optimizerD, G_losses, D_losses, emergency_path)
            print(f"💾 Emergency checkpoint saved: {emergency_path}")
            continue

    # Final cleanup and model saving
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

    torch.save(netG.state_dict(), f"{CONFIG['OUTPUT_DIR']}/netG_final.pth")
    torch.save(netD.state_dict(), f"{CONFIG['OUTPUT_DIR']}/netD_final.pth")
    print("💾 Saved final models")
    print("✅ GAN Training Complete!")

    return netG, netD, G_losses, D_losses

# Run training
generator, discriminator, G_losses, D_losses = run_gan_training(CONFIG)

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

✅ Checkpoint loaded successfully from epoch 179
Resuming from epoch 179
✅ Starting from epoch 179
Starting Training Loop...


...

✅ Checkpoint saved successfully for epoch 199


Epoch 200/200:  92%|█████████▏| 81/88 [02:05<00:15,  2.17s/it]

✅ Checkpoint saved successfully for epoch 199


Epoch 200/200: 100%|██████████| 88/88 [02:09<00:00,  1.47s/it]



📊 Epoch 200 Summary:
   Avg G Loss: 489.1017
   Avg D Loss: -139.5694
   Avg Wasserstein: 231.5773
📉 Learning rate reduced to: 8.589934592000007e-06
🖼️  Saved generated images for epoch 200
💾 Saved model weights for epoch 200
✅ Checkpoint saved successfully for epoch 199
💾 Epoch 200 completed. Checkpoint saved.
💾 Saved final models
✅ GAN Training Complete!
